# Python Assignment: ARIMA Model Forecasting with Walk-Forward Validation

This assignment delves into advanced time series forecasting by combining the robust ARIMA (AutoRegressive Integrated Moving Average) model with the critical evaluation technique of walk-forward validation. You will prepare time series data, select appropriate ARIMA orders, implement a walk-forward forecasting loop, and thoroughly evaluate your model's real-world performance. This approach is essential for building reliable forecasting systems.

## Part 1: Data Preparation and Exploratory Analysis (30 points)

You will start by loading or generating a suitable time series dataset and conducting initial exploratory analysis, including checking for stationarity.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings

warnings.filterwarnings("ignore") # Suppress warnings for cleaner output
np.random.seed(42) # for reproducibility

# 1.1 Load/Generate Time Series Data
#    Load a real-world time series dataset (e.g., from `statsmodels.datasets` or Kaggle).
#    *Recommendation: Use `statsmodels.datasets.co2.load().data` or `statsmodels.datasets.airquality.load().data` after some cleaning.*
#    Alternatively, generate a synthetic one with trend and seasonality.
#    Ensure your data is a Pandas Series with a DatetimeIndex.
#    Select a single column if loading a DataFrame with multiple.

# Example for CO2 data (uncomment and use if you prefer this)
try:
    from statsmodels.datasets import co2
    data = co2.load().data
    ts_data = data.resample('MS').mean().ffill() # Resample to monthly, forward fill missing
    ts_data = ts_data['co2'] # Select the 'co2' column
    print("Loaded CO2 dataset.")
except Exception as e:
    print(f"Could not load CO2 data, generating synthetic data: {e}")
    # Fallback to synthetic data if real data loading fails
    n_points = 200 # Roughly 16 years of monthly data
    dates = pd.date_range(start='2000-01-01', periods=n_points, freq='MS')

    trend = np.linspace(0, 50, n_points) # Increasing trend
    seasonality = 10 * np.sin(np.linspace(0, 3 * np.pi, n_points)) # Annual seasonality
    noise = np.random.normal(0, 2, n_points)

    ts_data = pd.Series(trend + seasonality + noise, index=dates)

print("Time Series Data Head:\n", ts_data.head())
print("Time Series Data Info:")
ts_data.info()

# 1.2 Plot the Raw Time Series
#    Visualize the raw time series to observe initial trends, seasonality, and noise.

plt.figure(figsize=(15, 6))
# TODO: Plot the time series
# plt.plot(ts_data)
# plt.title("Raw Time Series Data")
# plt.xlabel("Date")
# plt.ylabel("Value")
# plt.grid(True)
# plt.show()

# 1.3 Check for Stationarity (ADF Test)
#    Perform the Augmented Dickey-Fuller (ADF) test on the `ts_data`.
#    Interpret the p-value and critical values to determine if the series is stationary.

print("\n--- ADF Test on Raw Data ---")
# TODO: Perform ADF test
# result = adfuller(ts_data.dropna())
# print(f'ADF Statistic: {result[0]:.4f}')
# print(f'p-value: {result[1]:.4f}')
# print('Critical Values:')
# for key, value in result[4].items():
#     print(f'  {key}: {value:.4f}')
# if result[1] <= 0.05:
#     print("Conclusion: Data is likely stationary (reject H0).")
# else:
#     print("Conclusion: Data is likely non-stationary (fail to reject H0).")

# 1.4 Differencing for Stationarity (if needed)
#    If the ADF test indicates non-stationarity, apply first-order differencing.
#    Re-run the ADF test on the differenced series and plot it.
#    Keep track of the number of differences (`d` parameter for ARIMA).

d_order = 0 # Initialize differencing order
stationary_ts = ts_data.copy()

if result[1] > 0.05: # If not stationary, apply differencing
    print("\n--- Applying First-Order Differencing ---")
    # TODO: Apply differencing
    # stationary_ts = ts_data.diff().dropna()
    # d_order = 1

    plt.figure(figsize=(15, 6))
    # TODO: Plot differenced series
    # plt.plot(stationary_ts)
    # plt.title("Differenced Time Series Data (d=1)")
    # plt.xlabel("Date")
    # plt.ylabel("Differenced Value")
    # plt.grid(True)
    # plt.show()

    print("\n--- ADF Test on Differenced Data ---")
    # TODO: Re-run ADF test on differenced data
    # result_diff = adfuller(stationary_ts)
    # print(f'ADF Statistic: {result_diff[0]:.4f}')
    # print(f'p-value: {result_diff[1]:.4f}')
    # if result_diff[1] <= 0.05:
    #     print("Conclusion: Data is now likely stationary.")
    # else:
    #     print("Conclusion: Data still non-stationary, consider more differencing or seasonal differencing.")

print(f"Selected differencing order (d): {d_order}")

# 1.5 Plot ACF and PACF
#    Plot the Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) of the *stationary* series.
#    These plots will help you determine the `p` (AR order) and `q` (MA order) parameters for ARIMA.

plt.figure(figsize=(16, 8))
ax1 = plt.subplot(211)
# TODO: Plot ACF
# plot_acf(stationary_ts, ax=ax1, lags=40, title='Autocorrelation Function (ACF)')
ax2 = plt.subplot(212)
# TODO: Plot PACF
# plot_pacf(stationary_ts, ax=ax2, lags=40, title='Partial Autocorrelation Function (PACF)')
# plt.tight_layout()
# plt.show()


## Part 2: ARIMA Model Selection (20 points)

Based on the ACF and PACF plots, you'll propose ARIMA orders. For a tougher challenge, you can also explore `pmdarima`.

In [ ]:
# 2.1 Manual ARIMA Order Selection
#    Based on the ACF and PACF plots from Part 1.5, propose values for `p` and `q`.
#    Remember:
#    - `p`: Lag order of AR part (from PACF, where it cuts off)
#    - `q`: Lag order of MA part (from ACF, where it cuts off)
#    The `d` order was determined in Part 1.4.

p_order = 0 # TODO: Your proposed p value
q_order = 0 # TODO: Your proposed q value

print(f"Proposed ARIMA orders: ({p_order}, {d_order}, {q_order})")

### Justification for p and q:
*(Write your reasoning here for choosing `p` and `q` based on the ACF/PACF plots. E.g., "PACF shows a significant spike at lag X and then cuts off, suggesting p=X. ACF tails off, suggesting AR component. ACF shows a significant spike at lag Y and then cuts off, suggesting q=Y. PACF tails off, suggesting MA component.")*


# 2.2 (Optional - Bonus) Auto-ARIMA for Optimal Orders
#    Install `pmdarima` (`!pip install pmdarima`).
#    Use `pmdarima.auto_arima` to find the optimal ARIMA orders for your `ts_data`.
#    Compare these auto-detected orders with your manual selection.

try:
    from pmdarima import auto_arima
    print("\n--- Running Auto-ARIMA (Optional) ---")
    # TODO: Run auto_arima
    # auto_model = auto_arima(ts_data, seasonal=False, # Set seasonal=True if you suspect seasonality not handled by differencing
    #                       trace=True, error_action='ignore', suppress_warnings=True,
    #                       stepwise=True)
    # print("Auto-ARIMA Best Parameters:", auto_model.order)
    # print(auto_model.summary())
except ImportError:
    print("pmdarima not installed. Skipping Auto-ARIMA.")


## Part 3: Walk-Forward Validation Implementation (35 points)

This is the core of the assignment. You will implement a walk-forward validation scheme to evaluate your ARIMA model's performance on a rolling basis.

In [ ]:
# 3.1 Define Walk-Forward Validation Parameters
#    - `n_train_initial`: The number of observations in the initial training set.
#    - `n_forecast_steps`: The number of steps to forecast in each iteration (typically 1 for true walk-forward).

n_train_initial = int(len(ts_data) * 0.7) # Start with 70% of data for initial training
n_forecast_steps = 1 # Forecast one step at a time

print(f"Total data points: {len(ts_data)}")
print(f"Initial training set size: {n_train_initial}")
print(f"Number of forecast iterations: {len(ts_data) - n_train_initial}")

history = [x for x in ts_data[:n_train_initial]] # Initial training data
predictions = list() # To store all 1-step forecasts
actuals = list() # To store all actual values corresponding to forecasts

print("\n--- Starting Walk-Forward Validation ---")

# 3.2 Implement the Walk-Forward Validation Loop
#    Iterate through the remaining data points (the test set).
#    In each iteration:
#    - Train an ARIMA model using the `history` (current training data) and your chosen `(p, d, q)` orders.
#    - Make a 1-step forecast (or `n_forecast_steps`).
#    - Store the forecast and the actual value.
#    - Append the actual value from the test set to `history` to expand the training window for the next iteration.

for t in range(n_train_initial, len(ts_data)):
    # TODO: Train ARIMA model
    # model = ARIMA(history, order=(p_order, d_order, q_order))
    # model_fit = model.fit()

    # TODO: Make forecast
    # yhat = model_fit.forecast(steps=n_forecast_steps)[0] # Get the first forecast for 1-step
    # predictions.append(yhat)

    # Get actual value
    # obs = ts_data[t]
    # actuals.append(obs)

    # Add actual observation to history for next iteration
    # history.append(obs)

    # Print progress (optional)
    # if (t - n_train_initial) % 50 == 0 or t == len(ts_data) - 1:
    #     print(f'  Iteration {t - n_train_initial + 1}/{len(ts_data) - n_train_initial}: Predicted={yhat:.2f}, Actual={obs:.2f}')

print("\nWalk-Forward Validation Complete.")


## Part 4: Evaluation and Visualization (10 points)

Visualize the forecasts against actuals and calculate key performance metrics.

In [ ]:
# Convert lists to Pandas Series for easier plotting and indexing
forecast_index = ts_data.index[n_train_initial:]
predictions_series = pd.Series(predictions, index=forecast_index)
actuals_series = pd.Series(actuals, index=forecast_index)

# 4.1 Plot Actual vs. Forecasted Values
#    Plot the original time series, then overlay the forecasted values and the actual test set values.

plt.figure(figsize=(15, 7))
# TODO: Plot original data (up to test set start)
# plt.plot(ts_data[:n_train_initial], label='Training Data', color='blue')
# TODO: Plot actuals from the test set
# plt.plot(actuals_series, label='Actual Test Data', color='green')
# TODO: Plot predictions from walk-forward validation
# plt.plot(predictions_series, label='ARIMA Forecasts (Walk-Forward)', color='red', linestyle='--')

# plt.title("ARIMA Walk-Forward Forecast vs. Actuals")
# plt.xlabel("Date")
# plt.ylabel("Value")
# plt.legend()
# plt.grid(True)
# plt.show()

# 4.2 Calculate Performance Metrics
#    Calculate Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE) between the actuals and forecasts.
#    (Optional: Calculate Mean Absolute Percentage Error (MAPE) if applicable to your data).

# TODO: Calculate RMSE
rmse = # ...
# TODO: Calculate MAE
mae = # ...

print(f"\n--- Forecast Evaluation Metrics ---")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

# Optional: MAPE calculation
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# mape = mean_absolute_percentage_error(actuals_series, predictions_series)
# print(f"MAPE: {mape:.2f}%")


## Part 5: Reflection and Advanced Topics (5 points)

Reflect on the assignment and consider advanced aspects of time series forecasting.

### Your Answers to Reflection Questions:

1.  **Explain the primary benefits of using walk-forward validation for time series forecasting compared to a single train-test split.** What are its drawbacks (e.g., computational cost)?

    _(Your answer here)_

2.  **Based on your model's performance, what are some potential next steps you would take to improve the forecasting accuracy?** (Consider different models, feature engineering, etc.)

    _(Your answer here)_

3.  **When might a simple ARIMA model be insufficient for a time series, and what other factors or models would you consider then?** (e.g., multiple seasonalities, exogenous variables, non-linear patterns).

    _(Your answer here)_

4.  **(Bonus for Seasonal Data):** If your data showed strong *seasonal* patterns that weren't fully captured by differencing, how would you extend the ARIMA model (i.e., what type of model and parameters would you look into)?

    _(Your answer here)_


## Deliverables:

1.  This completed Jupyter Notebook (`arima_walk_forward_assignment.ipynb`) with all code cells executed and reflection questions answered.
2.  Ensure all plots are clearly visible and well-labeled within the notebook.